# <p style="text-align:center;">**OPTICOST**</p>
## <p style="text-align:center;">Sistem Mengoptimalkan Perhitungan Biaya Produksi dengan Python</p>
---

FILE YANG DIBUTUHKAN:
>- Data Koran DP
>- Koran Dyeing 2024
>- Koran Finishing 2024 new
>- Koran Pretreatment 2024
>- Koran Printing dan Digital Print 2024
>- Packing Area Note Dyeing_Printing -Hasil Produksi

# Penggabungan Data

In [1]:
import pandas as pd
import os
import re


In [2]:
def convert_data_types(df):
    # Kolom-kolom yang ingin diubah ke tipe float
    columns_to_convert_to_float = ['Kecepatan', 'Panjang_In', 'Panjang_Out_BQ', 'Panjang_Out_BS', 'Total_Panjang_Out', 'Durasi', 'Shift Number','Month Number']
    for col in columns_to_convert_to_float:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Kolom-kolom yang ingin diubah ke tipe datetime
    columns_to_convert_to_datetime = ['Start', 'Finish']
    for col in columns_to_convert_to_datetime:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce',format='%H:%M:%S').dt.time

    if 'Tanggal' in df.columns:
        df['Tanggal'] = pd.to_datetime(df['Tanggal'], errors='coerce', format='%m/%d/%Y %I:%M:%S %p' )

    return df

# Daftar file Excel yang akan digabungkan
file_list = [
    'Koran Pretreatment 2024.xlsx',
    'Koran Dyeing 2024.xlsx',
    'Koran Printing dan Digital Print 2024..xlsx',
    'Koran Finishing 2024 new.xlsx'
]

# Daftar untuk menyimpan DataFrame
df_list = []

# Membaca header dari file pertama
try:
    first_file = file_list[0]
    if not os.path.exists(first_file):
        raise FileNotFoundError(f'File tidak ditemukan: {first_file}')
    
    # Membaca file pertama untuk mendapatkan header
    df_first = pd.read_excel(first_file, sheet_name='Produksi', header=1)
    header = df_first.columns
    df_list.append(df_first[2:].reset_index(drop=True))  # Menyimpan DataFrame pertama dengan header yang benar
    print(f'Berhasil membaca file pertama: {first_file}')
    
except Exception as e:
    print(f'Error saat membaca file pertama {first_file}: {e}')

# Membaca file-file lainnya
for file in file_list[1:]:
    try:
        # Cek apakah file ada
        if not os.path.exists(file):
            print(f'File tidak ditemukan: {file}')
            continue
        
        # Membaca file tanpa header
        df = pd.read_excel(file, sheet_name='Produksi', header=None)
        
        # Terapkan header yang diambil dari file pertama
        df.columns = header
        df = df[2:].reset_index(drop=True)
        
        df_list.append(df)
        print(f'Berhasil membaca file: {file}')
        
    except Exception as e:
        print(f'Error saat membaca file {file}: {e}')

# Menggabungkan semua DataFrame menjadi satu jika ada DataFrame yang berhasil dibaca
if df_list:
    try:
        # Menggabungkan DataFrame
        combined_df = pd.concat(df_list, ignore_index=True)

        # Mengubah tipe data kolom yang diinginkan
        combined_df = convert_data_types(combined_df)

        # Menyimpan DataFrame gabungan ke file Excel baru
        combined_df.to_excel('koran_DP_gabungan.xlsx', index=False)

        print('File Excel berhasil digabungkan dan disimpan sebagai koran_DP_gabungan.xlsx')
    except Exception as e:
        print(f'Error saat menggabungkan DataFrame: {e}')
else:
    print('Tidak ada file yang berhasil dibaca dan digabungkan.')


Berhasil membaca file pertama: Koran Pretreatment 2024.xlsx
Berhasil membaca file: Koran Dyeing 2024.xlsx
Berhasil membaca file: Koran Printing dan Digital Print 2024..xlsx
Berhasil membaca file: Koran Finishing 2024 new.xlsx
File Excel berhasil digabungkan dan disimpan sebagai koran_DP_gabungan.xlsx


In [3]:
koran = pd.read_excel('koran_DP_gabungan.xlsx')
koran['Month Number'] = koran['Month Number'].fillna(0).apply(lambda x: int(x))
koran ['No_Order'].astype(str)
koran ['No_Order']=koran ['No_Order'].str.upper().str.strip()
# koran.head(3)



## Fungsi filter

In [4]:
def filter_output(koran, nama_kolom, threshold):
    filtered_koran = koran[koran[nama_kolom] > threshold]
    return filtered_koran


# Data Koran (khusus produksi)

In [5]:
data_koran =filter_output (koran, 'Total_Panjang_Out', 0)
#data_koran.to_excel('#Data koran khusus produksi.xlsx')
# data_koran.head(3)


In [6]:
jumlah_baris = len(data_koran)
total_durasi = data_koran['Durasi'].sum()
total_output = data_koran['Total_Panjang_Out'].sum()
max_date = data_koran['Tanggal'].max().strftime('%Y-%m-%d')

# Membuat DataFrame untuk tabel
summary_data = {
    'Keterangan': ['Jumlah data koran', 'Total Durasi', 'Total Output', 'Max Date'],
    'Nilai': [f'{jumlah_baris} baris', f'{total_durasi} menit', f'{total_output} meter', max_date]
}
summary_df = pd.DataFrame(summary_data)
summary_df


,Keterangan,Nilai
0,Jumlah data koran,36552 baris
1,Total Durasi,1112712.0 menit
2,Total Output,47081906.2 meter
3,Max Date,2024-09-09


## SPP tidak terformat

In [7]:
pattern = r'^[\w]{1,3}/\d{4}/\d{4}$'
SPP_non_matching = data_koran[~data_koran['No_Order'].str.match(pattern,na=False)]
SPP_non_matching.to_excel('01. SPP tidak terformat.xlsx')
SPP_non_matching


,Mesin,Jenis Order,Tanggal,Week,Bulan,Shift,Grup,No_Order,Material,No_Kereta,...,Durasi,Keterangan_Waktu,Jumlah Hari,Data_Kapasitas,Kecepatan_Ideal,Kapasitas_Pengerjaan,Persentase_Panjang_Out,Shift Number,Month Number,Area
49,OSTHOFF,Dyeing,2024-01-03,Week 01,Januari,Siang,A,D/2023/1927 II,TR 96 56 63,1.1.3,...,5.0,Produksi,31.0,3464208.0,77.603226,388.016129,0.814399,2.0,1,pretreatment
225,CBR,Dyeing,2024-01-03,Week 01,Januari,SIANG,A,D/2023/1927 L,TR 965663,2.1.12,...,25.0,Produksi,31.0,2618784.0,58.664516,1466.612903,1.065721,2.0,1,pretreatment
226,CBR,Dyeing,2024-01-03,Week 01,Januari,SIANG,A,D/2023/1927 L,TR 965663,2.2.11,...,25.0,Produksi,31.0,2618784.0,58.664516,1466.612903,1.092995,2.0,1,pretreatment
356,CMR,Dyeing,2024-01-03,Week 01,Januari,Siang,A,GT/2023/0185 - GT/2023/0188,PC 1107663,1.1.1,...,1.0,Produksi,31.0,2920320.0,65.419355,65.419355,0.611440,2.0,1,pretreatment
483,WD 2,Dyeing,2024-01-03,Week 01,Januari,SIANG,A,GT/2023/0185 - GT/2023/0188,PC 1107663,1.1.1,...,1.0,Produksi,31.0,3103488.0,69.522581,69.522581,0.575353,2.0,1,pretreatment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102437,ST,Digital Print,2024-06-18,Week 25,Juni,Pagi,B,DP/2024/0172T,VISCOSE 84 70 63,1.1,...,10.0,Produksi,30.0,828360.0,19.175000,191.750000,0.636245,1.0,6,finishing
102440,ST,Dyeing,2024-06-18,Week 25,Juni,Siang,A,D/2024/0148T,Ry Twill 2/1 96 66 63,1.1,...,10.0,Produksi,30.0,828360.0,19.175000,191.750000,0.714472,2.0,6,finishing
102450,ST,Digital Print,2024-06-19,Week 25,Juni,Siang,A,DP/2024/0125 T,VISCOSE 847063,1.1,...,10.0,Produksi,30.0,828360.0,19.175000,191.750000,0.771838,2.0,6,finishing
102451,ST,Digital Print,2024-06-19,Week 25,Juni,Siang,A,DP/2024/0126 T,VISCOSE 847063,1.1,...,10.0,Produksi,30.0,828360.0,19.175000,191.750000,0.458931,2.0,6,finishing


In [8]:
jumlah_baris = len(SPP_non_matching)
total_durasi = SPP_non_matching['Durasi'].sum()
total_output = SPP_non_matching['Total_Panjang_Out'].sum()

# Membuat DataFrame untuk tabel
summary_data = {
    'Keterangan': ['Jumlah SPP non matching', 'Total Durasi', 'Total Output'],
    'Nilai': [jumlah_baris, total_durasi, total_output],
    'Satuan': ['baris','menit','meter']
}
summary_df = pd.DataFrame(summary_data)

# Menampilkan tabel
summary_df


,Keterangan,Nilai,Satuan
0,Jumlah SPP non matching,1415.0,baris
1,Total Durasi,38662.0,menit
2,Total Output,1290475.8,meter


In [9]:
Rekap_non_maching = SPP_non_matching.groupby('Jenis Order').agg( # type: ignore
                                                Durasi  = ('Durasi',"sum"),
                                                Qty = ('Total_Panjang_Out','sum')).sort_values(by='Qty',ascending = False)
# Rekap_non_maching


## SPP Terformat

In [10]:
pattern = r'^[\w]{1,3}/\d{4}/\d{4}$'
SPP_matching = data_koran[data_koran['No_Order'].str.match(pattern,na=False)]
SPP_matching.head(3)


,Mesin,Jenis Order,Tanggal,Week,Bulan,Shift,Grup,No_Order,Material,No_Kereta,...,Durasi,Keterangan_Waktu,Jumlah Hari,Data_Kapasitas,Kecepatan_Ideal,Kapasitas_Pengerjaan,Persentase_Panjang_Out,Shift Number,Month Number,Area
3,OSTHOFF,Printing,2024-01-02,Week 01,Januari,Siang,A,P/2023/2300,COTTON 120 60 63,5.1.7,...,20.0,Produksi,31.0,3464208.0,77.603226,1552.064516,1.153947,2.0,1,pretreatment
4,OSTHOFF,Printing,2024-01-02,Week 01,Januari,Siang,A,P/2023/2300,COTTON 120 60 63,5.2.7,...,20.0,Produksi,31.0,3464208.0,77.603226,1552.064516,1.155880,2.0,1,pretreatment
5,OSTHOFF,Printing,2024-01-02,Week 01,Januari,Siang,A,P/2023/2349,CD 120 60 63,1.1.6,...,20.0,Produksi,31.0,3464208.0,77.603226,1552.064516,0.994160,2.0,1,pretreatment


In [11]:
# SPP_matching.to_excel('_SPP terformat BENAR.xlsx')


In [12]:
jumlah_baris = len(SPP_matching)
durasi_min = SPP_matching['Durasi'].min()
output_min = SPP_matching['Total_Panjang_Out'].min()
total_durasi = SPP_matching['Durasi'].sum()
total_output = SPP_matching['Total_Panjang_Out'].sum()

# Membuat DataFrame untuk tabel
summary_data = {
    'Keterangan': ['Jumlah SPP non matching', 'Durasi Min', 'Output Min', 'Total Durasi', 'Total Output'],
    'Nilai': [jumlah_baris, durasi_min, output_min, total_durasi, total_output],
    'Satuan': ['baris','menit','meter','menit','meter']
}
summary_df = pd.DataFrame(summary_data)

# Menampilkan tabel
summary_df


,Keterangan,Nilai,Satuan
0,Jumlah SPP non matching,35137.0,baris
1,Durasi Min,0.0,menit
2,Output Min,0.5,meter
3,Total Durasi,1074050.0,menit
4,Total Output,45791430.4,meter


## durasi =0

In [13]:
# SPP_matching =pd.read_excel('_SPP terformat BENAR.xlsx')
data_bulan = SPP_matching[(SPP_matching['Durasi'] == 0)]
data_bulan.head(2)


,Mesin,Jenis Order,Tanggal,Week,Bulan,Shift,Grup,No_Order,Material,No_Kereta,...,Durasi,Keterangan_Waktu,Jumlah Hari,Data_Kapasitas,Kecepatan_Ideal,Kapasitas_Pengerjaan,Persentase_Panjang_Out,Shift Number,Month Number,Area
6376,CMR,Printing Test,2024-02-27,Week 09,Februari,Pagi,B,PT/2024/0006,TC 1107663,1.1.1,...,0.0,Produksi,29.0,2920320.0,69.931034,0.0,NaN,1.0,2,pretreatment
14910,CBR,Printing,2024-06-04,Week 23,Juni,PAGI,B,P/2024/0538,CD 9472125,2.2.7,...,0.0,Produksi,30.0,2618784.0,60.620000,0.0,NaN,1.0,6,pretreatment


In [14]:
data_bulan.to_excel('_durasi 0 memiliki output .xlsx')


In [15]:
bulan = ['1','2','3','4','5','6','7','8','9','10','11','12']
SPP_matching.loc[:,'Month Number'] = SPP_matching['Month Number']
SPP_matching['Month Number'] = SPP_matching['Month Number'].astype(str)
data_olah = pd.DataFrame()

for i in bulan:
    data_bulan = SPP_matching[(SPP_matching['Month Number'].str.match(i,na=False)) &
                              (SPP_matching['Durasi'] > 0)]
    data_olah = pd.concat([data_olah,data_bulan],ignore_index=True)
    data_olah[['Tanggal','Bulan','Jenis Order','No_Order','Material','Keterangan_Waktu','Total_Panjang_Out','Durasi','Keterangan_Waktu','Area']]
data_olah.head(2)


C:\Users\irma.manik\AppData\Local\Temp\ipykernel_23568\1864598507.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SPP_matching['Month Number'] = SPP_matching['Month Number'].astype(str)


,Mesin,Jenis Order,Tanggal,Week,Bulan,Shift,Grup,No_Order,Material,No_Kereta,...,Durasi,Keterangan_Waktu,Jumlah Hari,Data_Kapasitas,Kecepatan_Ideal,Kapasitas_Pengerjaan,Persentase_Panjang_Out,Shift Number,Month Number,Area
0,OSTHOFF,Printing,2024-01-02,Week 01,Januari,Siang,A,P/2023/2300,COTTON 120 60 63,5.1.7,...,20.0,Produksi,31.0,3464208.0,77.603226,1552.064516,1.153947,2.0,1,pretreatment
1,OSTHOFF,Printing,2024-01-02,Week 01,Januari,Siang,A,P/2023/2300,COTTON 120 60 63,5.2.7,...,20.0,Produksi,31.0,3464208.0,77.603226,1552.064516,1.155880,2.0,1,pretreatment


In [16]:
data_olah['key'] = data_olah['Area']+data_olah['Month Number']
data_olah.head(2)


,Mesin,Jenis Order,Tanggal,Week,Bulan,Shift,Grup,No_Order,Material,No_Kereta,...,Keterangan_Waktu,Jumlah Hari,Data_Kapasitas,Kecepatan_Ideal,Kapasitas_Pengerjaan,Persentase_Panjang_Out,Shift Number,Month Number,Area,key
0,OSTHOFF,Printing,2024-01-02,Week 01,Januari,Siang,A,P/2023/2300,COTTON 120 60 63,5.1.7,...,Produksi,31.0,3464208.0,77.603226,1552.064516,1.153947,2.0,1,pretreatment,pretreatment1
1,OSTHOFF,Printing,2024-01-02,Week 01,Januari,Siang,A,P/2023/2300,COTTON 120 60 63,5.2.7,...,Produksi,31.0,3464208.0,77.603226,1552.064516,1.155880,2.0,1,pretreatment,pretreatment1


In [17]:
data_olah.to_excel('_Koran DP_layak diolah.xlsx')


# DATA OLAH 

In [18]:
import pandas as pd
# data_olah = pd.read_excel('_Koran DP_layak diolah.xlsx')
print(f'''jumlah SPP non matching = {len(data_olah)} baris,
output = {data_olah['Total_Panjang_Out'].sum()} meter, 
durasi min = {data_olah['Durasi'].min()} menit,
output min = {data_olah['Total_Panjang_Out'].min()} meter''')


jumlah SPP non matching = 35032 baris,
output = 45755808.6 meter, 
durasi min = 0.9999999999998366 menit,
output min = 0.5 meter


# Penilaian cost Produksi

In [19]:
data_olah_sorted = data_olah.sort_values(by='Month Number')
data = data_olah_sorted.groupby(['key','Month Number','Area','No_Order','Material','Mesin','Jenis Order']).agg(
                                Durasi = ('Durasi','sum'),
                                qty_meter = ('Total_Panjang_Out','sum')).reset_index()
data.to_excel('02. Rekap Koran_group 2024.xlsx')
data['Month Number'] = data['Month Number'].astype(str)
data.head(3)



,key,Month Number,Area,No_Order,Material,Mesin,Jenis Order,Durasi,qty_meter
0,dyeing1,1,dyeing,BD/2023/0074,JAC 05/012 1007065,Monfort,Booking Dyeing,1.0,7.0
1,dyeing1,1,dyeing,BD/2024/0007,JAC 1007065,PS 4,Booking Dyeing,2.0,14.0
2,dyeing1,1,dyeing,D/2023/0019,DB TENCEL 1337663,Monfort,Dyeing,1.0,30.0


# Data Area

In [20]:
area =data.groupby('key')[['Durasi','qty_meter']].sum().sort_values(by='Durasi',ascending=False)
total_data = data['Durasi'].sum()

area.head()


,Durasi,qty_meter
key,,
printing1,85978.0,1730816.03
pretreatment1,68859.0,4629294.00
printing7,61045.0,1096752.21
printing2,56991.0,1253987.81
printing8,51533.0,916189.40


## Import TMC

In [21]:
import pandas as pd
tmc= pd.read_excel('Data Koran DP.xlsx', sheet_name='TMC', header=0)
tmc['nilai'] = tmc['nilai'].astype(float)
tmc.head()


,Area,nilai,bulan
0,dyeing,1.214820e+09,1
1,finishing,7.427442e+08,1
2,pretreatment,1.911803e+09,1
3,printing,3.873009e+09,1
4,digital print,7.414865e+07,1


In [22]:
TMC =tmc.groupby('bulan')[['nilai']].sum().sort_values(by='bulan')
TMC


,nilai
bulan,
1,8.534277e+09
2,6.992719e+09
3,6.337862e+09
4,5.450908e+09
5,5.976646e+09
6,6.440713e+09
7,7.095897e+09
8,6.386904e+09


## Beban area

In [23]:
tmc_area =tmc['key']=tmc['Area']+tmc['bulan'].astype(str)
tmc_area =tmc[['key','nilai']]
tmc_area.head()


,key,nilai
0,dyeing1,1.214820e+09
1,finishing1,7.427442e+08
2,pretreatment1,1.911803e+09
3,printing1,3.873009e+09
4,digital print1,7.414865e+07


## Beban supporting

In [24]:
tmc_supp = tmc[tmc['Area'] == 'supporting'][['Area','bulan','nilai']]
tmc_supp.rename(columns={'bulan':'Month Number'}, inplace=True)
tmc_supp['Month Number'] = tmc_supp['Month Number'].astype(str)
print(f'beban supporting ={tmc_supp['nilai'].sum()}')
tmc_supp


beban supporting =4881080174.469425


,Area,Month Number,nilai
5,supporting,1,7.177530e+08
11,supporting,2,6.003961e+08
17,supporting,3,5.690663e+08
23,supporting,4,5.982843e+08
29,supporting,5,5.478900e+08
35,supporting,6,6.600605e+08
41,supporting,7,6.005799e+08
46,supporting,8,5.870501e+08


# Perhitungan Tarif

## Tarif supporting

In [25]:
tarif_supp = pd.merge(data,tmc_supp, on='Month Number', how='left')
tarif_supp['Total Durasi'] = tarif_supp.groupby('Month Number')['Durasi'].transform('sum')

tarif_supp['cost supporting Rp/menit'] = (tarif_supp['nilai']/tarif_supp['Total Durasi'])# diganti total durasi area IM > buat area IM

tarif_supp.rename(columns={'Area_x':'Area'}, inplace=True)
tarif_supp = tarif_supp[['key','Month Number','Area','No_Order','Material','Mesin','Jenis Order','Durasi','qty_meter','cost supporting Rp/menit']]
tarif_supp.to_excel('##tarif umum.xlsx')
total_supp = (tarif_supp ['Durasi']*tarif_supp ['cost supporting Rp/menit']).sum()  
print(f'Beban supporting (jan-jun)= {total_supp}')
tarif_supp.head(2)


Beban supporting (jan-jun)= 4881080174.469425


,key,Month Number,Area,No_Order,Material,Mesin,Jenis Order,Durasi,qty_meter,cost supporting Rp/menit
0,dyeing1,1,dyeing,BD/2023/0074,JAC 05/012 1007065,Monfort,Booking Dyeing,1.0,7.0,3438.370545
1,dyeing1,1,dyeing,BD/2024/0007,JAC 1007065,PS 4,Booking Dyeing,2.0,14.0,3438.370545


## Tarif Area

In [26]:
tarif_area=pd.merge(data,tmc_area, on='key', how='left')

tarif_area['Total Durasi'] = tarif_area.groupby('key')['Durasi'].transform('sum')
tarif_area['cost area Rp/menit'] = (tarif_area['nilai']/tarif_area['Total Durasi'])

tarif_area = tarif_area[['key','Month Number','Area','No_Order','Material','Mesin','Jenis Order','Durasi','qty_meter','cost area Rp/menit']]
total_area = (tarif_area ['Durasi']*tarif_area ['cost area Rp/menit']).sum()  
print(f'Beban area (jan-jun)= {total_area}')
tarif_area


Beban area (jan-jun)= 44064663820.01611


,key,Month Number,Area,No_Order,Material,Mesin,Jenis Order,Durasi,qty_meter,cost area Rp/menit
0,dyeing1,1,dyeing,BD/2023/0074,JAC 05/012 1007065,Monfort,Booking Dyeing,1.0,7.0,131360.292205
1,dyeing1,1,dyeing,BD/2024/0007,JAC 1007065,PS 4,Booking Dyeing,2.0,14.0,131360.292205
2,dyeing1,1,dyeing,D/2023/0019,DB TENCEL 1337663,Monfort,Dyeing,1.0,30.0,131360.292205
3,dyeing1,1,dyeing,D/2023/0028,TC ONLY 1337263,Monfort,Dyeing,31.0,2066.0,131360.292205
4,dyeing1,1,dyeing,D/2023/0192,PC 1337447,PS 4,Dyeing,1.0,10.0,131360.292205
...,...,...,...,...,...,...,...,...,...,...
20831,printing9,9,printing,P/2024/0948,CM 133 72 125,ZIMMER,Printing,30.0,725.0,NaN
20832,printing9,9,printing,P/2024/0951,C 1206063,HASPEL,Printing,55.0,1560.0,NaN
20833,printing9,9,printing,P/2024/0951,CD 120 60 63,ZIMMER,Printing,50.0,1560.0,NaN
20834,printing9,9,printing,PT/2024/0034,VISCOSE 84 70 63,ZIMMER,Printing Test,10.0,290.0,NaN


## Tarif Gabungan

In [27]:
# Define columns to merge on
merge_columns = ['key', 'Area','Month Number', 'No_Order', 'Material', 'Mesin', 'Jenis Order', 'Durasi', 'qty_meter']

# Perform the merge operation
tarif_gabungan = pd.merge(
    tarif_area,
    tarif_supp,
    on=merge_columns,
    how='left',
    suffixes=('_area', '_supp')  # Suffixes to distinguish columns if there are conflicts
)
tarif_gabungan['cost prod rp/menit'] = tarif_gabungan['cost area Rp/menit']+tarif_gabungan['cost supporting Rp/menit']
tarif_gabungan=tarif_gabungan [['key', 'Area', 'Month Number','No_Order', 'Material', 'Mesin', 'Jenis Order', 'Durasi', 'qty_meter','cost area Rp/menit','cost supporting Rp/menit','cost prod rp/menit']].reset_index()
# tarif_gabungan.to_excel('#01. tarif gab.xlsx')
#Display the merged DataFrame
#print(f"Shape of tarif_gabungan: {tarif_gabungan.shape}")
total_gab = (tarif_gabungan ['Durasi']*tarif_gabungan ['cost prod rp/menit']).sum()  #tarif gabungan rp/menits
print(f'Beban produksi = {total_gab}')
tarif_gabungan.head(2)


Beban produksi = 48945743994.485535


,index,key,Area,Month Number,No_Order,Material,Mesin,Jenis Order,Durasi,qty_meter,cost area Rp/menit,cost supporting Rp/menit,cost prod rp/menit
0,0,dyeing1,dyeing,1,BD/2023/0074,JAC 05/012 1007065,Monfort,Booking Dyeing,1.0,7.0,131360.292205,3438.370545,134798.66275
1,1,dyeing1,dyeing,1,BD/2024/0007,JAC 1007065,PS 4,Booking Dyeing,2.0,14.0,131360.292205,3438.370545,134798.66275


In [28]:
# Calculate total_tarif per menit
total_tarif = tarif_gabungan['cost prod rp/menit'] * tarif_gabungan['Durasi']
tarif_gabungan['Tot cost prod (rp/menit)'] =total_tarif

# Calculate tmc/meter
tarif_gabungan['tmc/meter'] = total_tarif / tarif_gabungan['qty_meter']
tarif_gabungan['Tot TMC(per SPP)'] = tarif_gabungan['tmc/meter'] *tarif_gabungan['qty_meter'] #total beban per spp

# Rename 'Area' to 'UNIT' (do this before selecting the columns)
tarif_gabungan.rename(columns={'Area': 'UNIT'}, inplace=True)

# Select the required columns
tarif_gabungan = tarif_gabungan[['key', 'UNIT', 'Month Number', 'No_Order', 'Material', 'Mesin', 'Jenis Order', 'Durasi', 'cost prod rp/menit','qty_meter', 'tmc/meter','Tot TMC(per SPP)']]

# Validate total_tarif
print(f'Validasi TMC = {total_tarif.sum()}')
tarif_gabungan.to_excel('#01. cost per SPP.xlsx')

# Display the first 2 rows of the updated DataFrame
tarif_gabungan.head(2)


Validasi TMC = 48945743994.485535


,key,UNIT,Month Number,No_Order,Material,Mesin,Jenis Order,Durasi,cost prod rp/menit,qty_meter,tmc/meter,Tot TMC(per SPP)
0,dyeing1,dyeing,1,BD/2023/0074,JAC 05/012 1007065,Monfort,Booking Dyeing,1.0,134798.66275,7.0,19256.951821,134798.662750
1,dyeing1,dyeing,1,BD/2024/0007,JAC 1007065,PS 4,Booking Dyeing,2.0,134798.66275,14.0,19256.951821,269597.325501


## Pivot cost

In [29]:
Rekap_kanban = tarif_gabungan.pivot_table(
                                            index=['No_Order','Month Number'],
                                            columns='UNIT',
                                            values=['qty_meter','tmc/meter','Tot TMC(per SPP)'],
                                            aggfunc='sum'
)
Rekap_kanban['Total (meter)'] = Rekap_kanban['qty_meter'].sum(axis=1)
Rekap_kanban['Total tarif'] = Rekap_kanban['tmc/meter'].sum(axis=1)
# Calculate 'Total beban' based on 'Total tarif'
Rekap_kanban['Total beban'] = Rekap_kanban['Tot TMC(per SPP)'].sum(axis=1)
# Display the DataFrame
Rekap_kanban.reset_index()

Rekap_kanban.to_excel('#02. Pivot cost.xlsx')
Rekap_kanban


Tot TMC(per SPP)                              \
UNIT                                dyeing     finishing  pretreatment   
No_Order     Month Number                                                
BD/2023/0009 1                         NaN  6.020501e+05           NaN   
BD/2023/0074 1                1.347987e+05           NaN           NaN   
BD/2023/0075 1                         NaN           NaN  6.240478e+04   
BD/2024/0001 1                         NaN           NaN  1.372905e+06   
BD/2024/0003 1                         NaN  2.006834e+04           NaN   
...                                    ...           ...           ...   
YD/2024/0027 4                         NaN           NaN  2.072014e+05   
             5                         NaN  1.545800e+06           NaN   
YD/2024/0028 4                1.509824e+06           NaN           NaN   
             5                         NaN  1.803433e+06           NaN   
YD/2024/0029 7                         NaN           NaN  1.250753e+06   

                                   qty_meter                                  \
UNIT                      printing    dyeing finishing pretreatment printing   
No_Order     Month Number                                                      
BD/2023/0009 1                 NaN       NaN    1165.0          NaN      NaN   
BD/2023/0074 1                 NaN       7.0       NaN          NaN      NaN   
BD/2023/0075 1                 NaN       NaN       NaN         40.0      NaN   
BD/2024/0001 1                 NaN       NaN       NaN       1748.0      NaN   
BD/2024/0003 1                 NaN       NaN       5.0          NaN      NaN   
...                            ...       ...       ...          ...      ...   
YD/2024/0027 4                 NaN       NaN       NaN        520.0      NaN   
             5                 NaN       NaN    2959.0          NaN      NaN   
YD/2024/0028 4                 NaN    1075.0       NaN          NaN      NaN   
             5                 NaN       NaN    3176.0          NaN      NaN   
YD/2024/0029 7                 NaN       NaN       NaN       2221.0      NaN   

                              tmc/meter                                     \
UNIT                             dyeing    finishing pretreatment printing   
No_Order     Month Number                                                    
BD/2023/0009 1                      NaN   516.781204          NaN      NaN   
BD/2023/0074 1             19256.951821          NaN          NaN      NaN   
BD/2023/0075 1                      NaN          NaN  1560.119534      NaN   
BD/2024/0001 1                      NaN          NaN  3946.168397      NaN   
BD/2024/0003 1                      NaN  4013.667353          NaN      NaN   
...                                 ...          ...          ...      ...   
YD/2024/0027 4                      NaN          NaN   398.464230      NaN   
             5                      NaN  1567.229492          NaN      NaN   
YD/2024/0028 4              1404.487627          NaN          NaN      NaN   
             5                      NaN  2640.930650          NaN      NaN   
YD/2024/0029 7                      NaN          NaN  1123.961859      NaN   

                          Total (meter)   Total tarif   Total beban  
UNIT                                                                 
No_Order     Month Number                                            
BD/2023/0009 1                   1165.0    516.781204  6.020501e+05  
BD/2023/0074 1                      7.0  19256.951821  1.347987e+05  
BD/2023/0075 1                     40.0   1560.119534  6.240478e+04  
BD/2024/0001 1                   1748.0   3946.168397  1.372905e+06  
BD/2024/0003 1                      5.0   4013.667353  2.006834e+04  
...                                 ...           ...           ...  
YD/2024/0027 4                    520.0    398.464230  2.072014e+05  
             5                   2959.0   1567.229492  1.545800e+06  
YD/20

## Hasil perhitungan tarif

In [30]:
Total_cost = Rekap_kanban.groupby('No_Order').agg({
    ('Total (meter)', ''): 'sum', 
    ('Total beban', ''): 'sum'
}).reset_index()

Total_cost.columns = ['No_Order', 'Total (meter)', 'Total beban']

# Menghapus kolom 'Month Number' karena tidak diperlukan dalam hasil akhir
# Total_cost.to_excel('#02. total_cost.xlsx')
Total_cost


,No_Order,Total (meter),Total beban
0,BD/2023/0009,1165.0,6.020501e+05
1,BD/2023/0074,7.0,1.347987e+05
2,BD/2023/0075,40.0,6.240478e+04
3,BD/2024/0001,1748.0,1.372905e+06
4,BD/2024/0003,5.0,2.006834e+04
...,...,...,...
3183,YD/2024/0025,4.0,2.962299e+04
3184,YD/2024/0026,4.0,2.962299e+04
3185,YD/2024/0027,3479.0,1.753001e+06
3186,YD/2024/0028,4251.0,3.313257e+06


# Hasil produksi DMS

In [31]:
import pandas as pd
hasilprod_dms1 = pd.read_excel("HASIL PRODUKSI DMS 2024.xlsx")
# print(f'total proacc = {hasilprod_acc['QTY'].sum()}')
hasilprod_dms1.head(2)


,ProductionOrderNo,DateOut,Construction,Unit,Color,Buyer,Grade,Balance,PackagingQuantity,Description
0,D/2023/1925,2024-01-02 10:22:04.386,CD RIPPLE / 88x58 / 57/58 / 40x40,DYEING,COL.CHOCO (A),PT PRAKASA TRIPUTRA SOLUSI,A1,104.24,1,UL
1,TP/2023/0185,2024-01-02 10:28:52.469,POLYESTER / 240x112 / 58/59 / 40x40,DYEING,ABU - ABU ( OPD DAN CAMAT ),PT EFRATA GARMINDO UTAMA,B,2.88,1,NaN


In [32]:
# Jika kolom 'QTY' dan 'QTY Pack' ada, lanjutkan dengan membuat pivot table
hasilprod_dms =hasilprod_dms1[~hasilprod_dms1['Description'].str.contains('UL',case = False, na = False)]
if 'Balance' in hasilprod_dms.columns and 'PackagingQuantity' in hasilprod_dms.columns:
    hasil_produksi_pivot = hasilprod_dms.pivot_table(
        index=['DateOut','ProductionOrderNo', 'Construction', 'Unit', 'Color', 'Buyer', 'Grade','Description'],
        values=['Balance', 'PackagingQuantity'],
        aggfunc={'Balance': 'sum', 'PackagingQuantity': 'sum'}
    )
    
# hasilprod_acc = hasilprod_dms[hasilprod_dms['KET'] == 'HASIL PRODUKSI']    
hasilprod_dms.head(2)


,ProductionOrderNo,DateOut,Construction,Unit,Color,Buyer,Grade,Balance,PackagingQuantity,Description
1,TP/2023/0185,2024-01-02 10:28:52.469,POLYESTER / 240x112 / 58/59 / 40x40,DYEING,ABU - ABU ( OPD DAN CAMAT ),PT EFRATA GARMINDO UTAMA,B,2.88,1,NaN
2,D/2023/1887,2024-01-02 10:56:21.628,TC ONLY / 110x60 / 59/60'' / 30x30,DYEING,ILLAWARA BLUE,PT. TRISCO TAILORED APPAREL MANUFACTURING,A,70.00,1,W1 S+T 3-4


### HASIL PRODUKSI AKHIR

In [33]:
import pandas as pd

pivotprod = hasilprod_dms.pivot_table(index=['DateOut','ProductionOrderNo','Construction','Unit'],                                           
                                    values=['Balance'],
                                    aggfunc='sum').reset_index()
pivotprod.rename(columns={'ProductionOrderNo':'NO SP',
                          'Construction': 'MATERIAL',
                          'Unit': 'UNIT',
                          'Balance' : 'QTY' },inplace=True)

pivotprod


,DateOut,NO SP,MATERIAL,UNIT,QTY
0,2024-01-02 10:28:52.469,TP/2023/0185,POLYESTER / 240x112 / 58/59 / 40x40,DYEING,2.88
1,2024-01-02 10:56:21.628,D/2023/1887,TC ONLY / 110x60 / 59/60'' / 30x30,DYEING,70.00
2,2024-01-02 10:56:21.789,D/2023/1887,TC ONLY / 110x60 / 59/60'' / 30x30,DYEING,49.00
3,2024-01-02 10:56:21.869,D/2023/1887,TC ONLY / 110x60 / 59/60'' / 30x30,DYEING,51.00
4,2024-01-02 10:56:21.959,D/2023/1887,TC ONLY / 110x60 / 59/60'' / 30x30,DYEING,146.00
...,...,...,...,...,...
41160,2024-10-07 09:23:31.017,D/2024/0764,CD RANKOUSHI / 82x58 / 59/60'' / CD32xCD32+20,DYEING,41.15
41161,2024-10-07 09:23:31.239,D/2024/0764,CD RANKOUSHI / 82x58 / 59/60'' / CD32xCD32+20,DYEING,88.70
41162,2024-10-07 09:23:31.455,D/2024/0764,CD RANKOUSHI / 82x58 / 59/60'' / CD32xCD32+20,DYEING,54.86
41163,2024-10-07 09:23:31.701,D/2024/0764,CD RANKOUSHI / 82x58 / 59/60'' / CD32xCD32+20,DYEING,9.14


## Price list

In [34]:
# Define columns to merge on
Total_cost.rename(columns={'No_Order': 'NO SP'}, inplace=True)
price_list =pd.merge(
    pivotprod,
    Total_cost,
    on='NO SP',
    how='left',
    )
price_list['cost/meter'] = price_list['Total beban']/price_list['QTY']
price_list['bulan']=price_list['DateOut'].dt.month_name()
price_list.to_excel('03. Pricelist_output GJ.xlsx')
print(f'Total produksi Juni= {price_list['QTY'].sum()}')

price_list [['DateOut', 'NO SP', 'MATERIAL', 'UNIT', 'QTY', 'Total beban', 'cost/meter', 'bulan']]


Total produksi Juni= 5523872.139999999


,DateOut,NO SP,MATERIAL,UNIT,QTY,Total beban,cost/meter,bulan
0,2024-01-02 10:28:52.469,TP/2023/0185,POLYESTER / 240x112 / 58/59 / 40x40,DYEING,2.88,NaN,NaN,January
1,2024-01-02 10:56:21.628,D/2023/1887,TC ONLY / 110x60 / 59/60'' / 30x30,DYEING,70.00,702391.786811,10034.168383,January
2,2024-01-02 10:56:21.789,D/2023/1887,TC ONLY / 110x60 / 59/60'' / 30x30,DYEING,49.00,702391.786811,14334.526261,January
3,2024-01-02 10:56:21.869,D/2023/1887,TC ONLY / 110x60 / 59/60'' / 30x30,DYEING,51.00,702391.786811,13772.387977,January
4,2024-01-02 10:56:21.959,D/2023/1887,TC ONLY / 110x60 / 59/60'' / 30x30,DYEING,146.00,702391.786811,4810.902649,January
...,...,...,...,...,...,...,...,...
41160,2024-10-07 09:23:31.017,D/2024/0764,CD RANKOUSHI / 82x58 / 59/60'' / CD32xCD32+20,DYEING,41.15,NaN,NaN,October
41161,2024-10-07 09:23:31.239,D/2024/0764,CD RANKOUSHI / 82x58 / 59/60'' / CD32xCD32+20,DYEING,88.70,NaN,NaN,October
41162,2024-10-07 09:23:31.455,D/2024/0764,CD RANKOUSHI / 82x58 / 59/60'' / CD32xCD32+20,DYEING,54.86,NaN,NaN,October
41163,2024-10-07 09:23:31.701,D/2024/0764,CD RANKOUSHI / 82x58 / 59/60'' / CD32xCD32+20,DYEING,9.14,NaN,NaN,October
